# The Hong Kong University of Science and Technology
# MSBD6000J: Spatial and Multimedia Databases
# Fall 2021 Assignment 2

### Student name: Mak Chun Wai, Michael
### HKUST account: cwmakah
### Student ID: 20801333

In [1]:
# import libraries needed
import math
import sys
import random
import numpy as np
import time
from math import sqrt
import pandas as pd

In [2]:
class Rect:
    '''
    Represents the MBR
    __init__: store 4 coordinates to represent x_low, y_low, x_high, y_high
    perimeter: calculate the perimeter of MBR by the 4 coordinates
    is_overlap: return a boolean to flag if another Rect is overlapping with self
    contain_rect: return a boolean to flag if another Rect is containing self
    has_point: return a boolean to flag if a point is inside the area of self
    '''
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2

    def perimeter(self):
        return 2 * (abs(self.x2 - self.x1) + abs(self.y2 - self.y1))

    def is_overlap(self, rect):
        if self.y1 > rect.y2 or self.y2 < rect.y1 or self.x1 > rect.x2 or self.x2 < rect.x1:
            return False
        else:
            return True
    
    def contain_rect(self, rect):
        return self.x1 < rect.x1 and self.y1 < rect.y1 and self.x2 > rect.x2 and self.y2 > rect.y2

    def has_point(self, point):
        return self.x1 <= point.x <= self.x2 and self.y1 <= point.y <= self.y2

In [3]:
class Point:
    '''
    Represent the 2D coordinates
    __init__: store the 2 coordinates as self.x and self.y
    '''
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [4]:
class Node(object):
    '''
    __init__: store bucket size n, maximum number of subtrees d, parent node, 
              MBR, children nodes (if have) & data points (if have)
    add_point: change a single input point to a list and pass to add_points
    add_points: add list of points into self.data_points, and then pass to update_MBR
    perimeter_increase_with_point: calculate the expansion of MBR if a new point is added into self
    perimeter: use Rect.perimeter to calculate the perimeter of self.MBR
    is_overflow: return a boolean to flag if self is overflow
    is_root: return a boolean to flag if self is the root node
    is_leaf: return a boolean to flag if self is a leaf node
    add_child_node: change a single input node to a list and pass to add_child_nodes
    add_child_nodes: add list of nodes into self.child_nodes, and then pass to update_MBR
    update_MBR: update the self.MBR by the min. and max. coordinates of self.child_nodes/self.data_points 
                (depend on type of self), and then check if the parent node also needs to update MBR
    get_points_MBR_perimeter: get the MBR perimeter for a list of points
    get_nodes_MBR_perimeter: get the MBR perimeter for a list of nodes
    get_min_dist: caculate the MINDIST for pruning strategies as in lecture notes
    get_min_max_dist: caculate the MINMAXDIST for pruning strategies as in lecture notes
    '''
    def __init__(self, n = 128, d = 2):
        self.n = n
        self.d = d
        self.parent_node = None
        self.MBR = Rect(-1, -1, -1, -1)
        # for index nodes
        self.child_nodes = []
        # for leaf nodes
        self.data_points = []
        

    def add_point(self, point):
        self.add_points([point])

    def add_points(self, points):
        for point in points:
            self.data_points.append(point)
        self.update_MBR()

    def perimeter_increase_with_point(self, point):
        x1 = point.x if point.x < self.MBR.x1 else self.MBR.x1
        y1 = point.y if point.y < self.MBR.y1 else self.MBR.y1
        x2 = point.x if point.x > self.MBR.x2 else self.MBR.x2
        y2 = point.y if point.y > self.MBR.y2 else self.MBR.y2
        return Rect(x1, y1, x2, y2).perimeter() - self.perimeter()

    def perimeter(self):
        return self.MBR.perimeter()

    def is_overflow(self):
        return (self.is_leaf() and len(self.data_points) > self.n) or \
               (not self.is_leaf() and len(self.child_nodes) > self.d)

    def is_root(self):
        return self.parent_node is None

    def is_leaf(self):
        return len(self.child_nodes) == 0

    def add_child_node(self, node):
        self.add_child_nodes([node])

    def add_child_nodes(self, nodes):
        for node in nodes:
            node.parent_node = self
            self.child_nodes.append(node)
        self.update_MBR()

    def update_MBR(self):
        if self.is_leaf():
            self.MBR.x1 = min([point.x for point in self.data_points])
            self.MBR.x2 = max([point.x for point in self.data_points])
            self.MBR.y1 = min([point.y for point in self.data_points])
            self.MBR.y2 = max([point.y for point in self.data_points])
        else:
            self.MBR.x1 = min([child.MBR.x1 for child in self.child_nodes])
            self.MBR.x2 = max([child.MBR.x2 for child in self.child_nodes])
            self.MBR.y1 = min([child.MBR.y1 for child in self.child_nodes])
            self.MBR.y2 = max([child.MBR.y2 for child in self.child_nodes])
        if self.parent_node and not self.parent_node.MBR.contain_rect(self.MBR):
            self.parent_node.update_MBR()

    def get_points_MBR_perimeter(points):
        x1 = min([point.x for point in points])
        x2 = max([point.x for point in points])
        y1 = min([point.y for point in points])
        y2 = max([point.y for point in points])
        return Rect(x1, y1, x2, y2).perimeter()

    def get_nodes_MBR_perimeter(nodes):
        x1 = min([node.MBR.x1 for node in nodes])
        x2 = max([node.MBR.x2 for node in nodes])
        y1 = min([node.MBR.y1 for node in nodes])
        y2 = max([node.MBR.y2 for node in nodes])
        return Rect(x1, y1, x2, y2).perimeter()
    
    def get_min_dist(self, pt):
        dx = min(abs(pt.x - self.MBR.x1), abs(pt.x - self.MBR.x2))
        dy = min(abs(pt.y - self.MBR.y1), abs(pt.y - self.MBR.y2))
        if self.MBR.x1 < pt.x < self.MBR.x2:
            return dy
        elif self.MBR.y1 < pt.y < self.MBR.y2:
            return dx
        else:
            return sqrt(dx ** 2 + dy ** 2)
        
    def get_min_max_dist(self, pt):
        if self.MBR.x1 < pt.x < self.MBR.x2:
            dx = max(abs(pt.x - self.MBR.x1), abs(pt.x - self.MBR.x2))
            dy = min(abs(pt.y - self.MBR.y1), abs(pt.y - self.MBR.y2))
        elif self.MBR.y1 < pt.y < self.MBR.y2:
            dx = min(abs(pt.x - self.MBR.x1), abs(pt.x - self.MBR.x2))
            dy = max(abs(pt.y - self.MBR.y1), abs(pt.y - self.MBR.y2))
        else:
            dx = min(abs(pt.x - self.MBR.x1), abs(pt.x - self.MBR.x2))
            dy = min(abs(pt.y - self.MBR.y1), abs(pt.y - self.MBR.y2))
        return sqrt(dx ** 2 + dy ** 2)

In [5]:
class RTree:
    '''
    __init__: store bucket size n, maximum number of subtrees d & the root node
    get_height: get the height of the tree
    update_info: update the global variables to get the number of non-leaf and leaf nodes, 
                 space utilization & number of subtrees overlapping
    insert_point: insert 1 point into the tree starting from the root node;
                  handle overflow if necessary (number of points > n or number of subtrees > d);
                  choose the best child node to insert the point if the current node is not a leaf node
    choose_best_child: choose the best child by minimum expansion after inserting a new point
    handle_overflow: split the nodes by split_leaf_node/split_internal_node (depend on the node type of current node);
                     empty data_points and append the child_nodes after splitting;
                     check if the parent node is overflow after splitting
    split_leaf_node: Sort by x-coordinate and y-coordinate to get the best set of splitting 
                     by minimum perimeter (for leaf node)
    split_internal_node: Sort by x-coordinate and y-coordinate to get the best set of splitting 
                         by minimum perimeter (for index node)
    search_NN: search the nearest neighbor of a query point by pruning subtrees by the 3 strategies in the process
    '''
    def __init__(self, n = 128, d = 2):
        self.n = n
        self.d = d
        self.root = Node(self.n, self.d)
    
    def get_height(self, node, cur_height):
        global max_height
        if self.root is None:
            max_height = 1
            
        if cur_height > max_height:
            max_height = cur_height
        if len(node.child_nodes) != 0:
            for child in node.child_nodes:
                self.get_height(child, cur_height + 1)
    
    def update_info(self, cur_node=None):
        global n_non_leaf
        global n_leaf
        global leaf_points_lst
        global n_overlap
        
        if cur_node is None:
            cur_node = self.root
            
        if len(cur_node.child_nodes) != 0:
            n_non_leaf += 1
            mbr_lst = []
            for child in cur_node.child_nodes:
                self.update_info(child)
                mbr_lst.append(child.MBR)
            for i in range(len(mbr_lst) - 1):
                for j in range(1, len(mbr_lst)):
                    if mbr_lst[i].is_overlap(mbr_lst[j]):
                        n_overlap += 1
        else: # Arrive leaf node
            n_leaf += 1
            leaf_points_lst.append(len(cur_node.data_points))
    
    def insert_point(self, point, cur_node=None):
        if cur_node is None:
            cur_node = self.root

        if cur_node.is_leaf():
            cur_node.add_point(point)
            # Handle overflow
            if cur_node.is_overflow():
                self.handle_overflow(cur_node)
        else:
            chosen_child = self.choose_best_child(cur_node, point)
            self.insert_point(point, cur_node = chosen_child)

    def choose_best_child(self, node, point):
        best_child = None
        best_perimeter = 0
        # Scan the child nodes
        for item in node.child_nodes:
            if node.child_nodes.index(item) == 0 or best_perimeter > item.perimeter_increase_with_point(point):
                best_child = item
                best_perimeter = item.perimeter_increase_with_point(point)
        return best_child

    def handle_overflow(self, node):
        node, new_node = self.split_leaf_node(node) if node.is_leaf() else self.split_internal_node(node)

        if node is self.root:
            self.root = Node(self.n, self.d)
            self.root.add_child_nodes([node, new_node])
        else:
            node.parent_node.add_child_node(new_node)
            node.parent_node.data_points = []
            if node.parent_node.is_overflow():
                self.handle_overflow(node.parent_node)

    def split_leaf_node(self, node):
        m = len(node.data_points)
        best_perimeter = float('inf')
        best_set_1 = []
        best_set_2 = []
        # Sort by x-coordinate
        all_point_sorted_by_x = sorted(node.data_points, key=lambda point: point.x)
        for i in range(int(0.4 * m), int(m * 0.6) + 1):
            list_point_1 = all_point_sorted_by_x[:i]
            list_point_2 = all_point_sorted_by_x[i:]
            temp_sum_perimeter = Node.get_points_MBR_perimeter(list_point_1) \
                                 + Node.get_points_MBR_perimeter(list_point_2)
            if best_perimeter > temp_sum_perimeter:
                best_perimeter = temp_sum_perimeter
                best_set_1 = list_point_1
                best_set_2 = list_point_2
        # Sort by y-coordinate
        all_point_sorted_by_y = sorted(node.data_points, key=lambda point: point.y)
        for i in range(int(0.4 * m), int(m * 0.6) + 1):
            list_point_1 = all_point_sorted_by_y[:i]
            list_point_2 = all_point_sorted_by_y[i:]
            temp_sum_perimeter = Node.get_points_MBR_perimeter(list_point_1) \
                                 + Node.get_points_MBR_perimeter(list_point_2)
            if best_perimeter > temp_sum_perimeter:
                best_perimeter = temp_sum_perimeter
                best_set_1 = list_point_1
                best_set_2 = list_point_2
        node.data_points = best_set_1
        node.update_MBR()
        new_node = Node(self.n, self.d)
        new_node.add_points(best_set_2)
        new_node.update_MBR()
        return node, new_node

    def split_internal_node(self, node):
        m = len(node.child_nodes)
        best_perimeter = float('inf')
        best_set_1 = []
        best_set_2 = []
        # Sort by x-coordinate
        all_node_sorted_by_x = sorted(node.child_nodes, key=lambda child: child.MBR.x1)
        for i in range(int(0.4 * m), int(m * 0.6) + 1):
            list_node_1 = all_node_sorted_by_x[:i]
            list_node_2 = all_node_sorted_by_x[i:]
            temp_sum_perimeter = Node.get_nodes_MBR_perimeter(list_node_1) \
                                 + Node.get_nodes_MBR_perimeter(list_node_2)
            if best_perimeter > temp_sum_perimeter:
                best_perimeter = temp_sum_perimeter
                best_set_1 = list_node_1
                best_set_2 = list_node_2
        # Sort by y-coordinate
        all_node_sorted_by_y = sorted(node.child_nodes, key=lambda child: child.MBR.y1)
        for i in range(int(0.4 * m), int(m * 0.6) + 1):
            list_node_1 = all_node_sorted_by_y[:i]
            list_node_2 = all_node_sorted_by_y[i:]
            temp_sum_perimeter = Node.get_nodes_MBR_perimeter(list_node_1) \
                                 + Node.get_nodes_MBR_perimeter(list_node_2)
            if best_perimeter > temp_sum_perimeter:
                best_perimeter = temp_sum_perimeter
                best_set_1 = list_node_1
                best_set_2 = list_node_2
        node.child_nodes = best_set_1
        node.update_MBR()
        new_node = Node(self.n, self.d)
        new_node.add_child_nodes(best_set_2)
        new_node.update_MBR()
        return node, new_node
    
    def search_NN(self, search_pt, cur_node = None):
        global rule1_pruned
        global rule2_pruned
        global rule3_pruned
        global ABL
        global nodes_visited
        global points_calculated
        global min_dist
        global NN
        global min_minmaxdist
        
        nodes_visited += 1
        
        if cur_node == None:
            cur_node = self.root
        
        if not cur_node.is_leaf():
            ABL_keep = []
            ABL_bool = []
            for child in cur_node.child_nodes:
                ABL.append((child, child.get_min_dist(search_pt), child.get_min_max_dist(search_pt)))
            # sort by MINDIST
            ABL = sorted(ABL, key=lambda x: x[1]) 
            # Pruning by rule 1
            for i in range(len(ABL)):
                for j in range(len(ABL)):
                    if ABL[i][1] > ABL[j][2]:
                        ABL_bool.append(False)
                    else:
                        ABL_bool.append(True)
                if sum(ABL_bool) == len(ABL_bool):
                    ABL_keep.append(ABL[i])
            rule1_pruned += (len(ABL) - len(ABL_keep))
            ABL = ABL_keep # pruning
            ABL_keep = []
            while len(ABL) != 0:
                min_minmaxdist = sorted(ABL, key=lambda x: x[2])[0][2]
                child = ABL.pop(0)[0]
                self.search_NN(search_pt, child)            
        else:
            ABL_keep = []
            # for root as the only node in tree
            if cur_node.is_root():
                for point in cur_node.data_points:
                    points_calculated += 1
                    dist = sqrt((point.x - search_pt.x) ** 2 + (point.y - search_pt.y) ** 2)
                    if dist < min_dist:
                        min_dist = dist
                        NN = point
                return
            # Pruning by rule 2
            points_keep = []
            if len(ABL) != 0:
                # get the min. MINMAXDIST among MBRs in ABL
                min_minmaxdist = sorted(ABL, key=lambda x: x[2])[0][2] 
            for point in cur_node.data_points:
                points_calculated += 1
                dist = sqrt((point.x - search_pt.x) ** 2 + (point.y - search_pt.y) ** 2)
                if dist <= min_minmaxdist:
                    points_keep.append(point)
                rule2_pruned += (len(cur_node.data_points) - len(points_keep))
                # Get NN and the L2 distance
                if dist < min_dist:
                    min_dist = dist
                    NN = point
            # Pruning by rule 3
            for branch in ABL:
                if branch[1] < min_dist:
                    ABL_keep.append(branch)
            rule3_pruned += (len(ABL) - len(ABL_keep))
            ABL = ABL_keep # pruning
            ABL_keep = []

In [6]:
# Get the MBR of given points
def get_mbr(data):
    x_low = min(data[:, 0])
    x_high = max(data[:, 0])
    y_low = min(data[:, 1])
    y_high = max(data[:, 1])
    return [x_low, x_high, y_low, y_high]

In [7]:
# Generate random points within a given space
def gen_rand_pt(space, pt_number):
    pt_lst = []
    while len(pt_lst) < pt_number:
        x = random.uniform(space[0], space[1])
        y = random.uniform(space[2], space[3])
        pt_lst.append((x, y))
    return pt_lst

In [8]:
# Load data from csv file
data = np.genfromtxt('AllPOI Simplified.csv', dtype="float", delimiter=',', encoding="utf-8-sig")

In [14]:
# Randomly generate 10 query points within the space
space = get_mbr(data)
rand_pt_lst = gen_rand_pt(space, 10)
rand_pt_lst

[(116.57297513956361, 40.83696241647718),
 (115.51991515952055, 40.18953817066641),
 (115.90869667162163, 39.89487920621066),
 (117.16139043718496, 39.67787445378875),
 (116.11805999422101, 40.373919675791726),
 (115.92769793735539, 40.39115273855126),
 (117.08518966777893, 39.87170008992925),
 (116.6687745873243, 40.08641152587313),
 (117.04144843803991, 39.445694745911574),
 (117.38652188040281, 40.59645912127938)]

In [29]:
rand_pt_df = pd.DataFrame(columns = ['x', 'y'])
for pt in rand_pt_lst:
    tmp_dict = {'x': '{:.4f}'.format(pt[0]), 'y': '{:.4f}'.format(pt[1])}
    rand_pt_df = rand_pt_df.append(tmp_dict, ignore_index = True)
rand_pt_df

,x,y
0,116.5730,40.8370
1,115.5199,40.1895
2,115.9087,39.8949
3,117.1614,39.6779
4,116.1181,40.3739
5,115.9277,40.3912
6,117.0852,39.8717
7,116.6688,40.0864
8,117.0414,39.4457
9,117.3865,40.5965


### R-tree (n = 128, d = 2)

In [26]:
# Insert the points in dataset and build the tree
start_time = time.time()
rtree = RTree(n = 128, d = 2)
for i in data:
    x = i[0]
    y = i[1]
    pt = Point(x, y)
    rtree.insert_point(pt)
time_used = time.time() - start_time
print(time_used)

124.25786423683167


In [27]:
# Initialize global parameters
n_non_leaf = 0
n_leaf = 0
leaf_points_lst = []
n_overlap = 0
max_height = 1

# Update the info of R-tree
rtree.update_info()
rtree.get_height(rtree.root, 1)

# Print result
print(f'Height of R-Tree index: {max_height}')
print(f'Number of non-leaf nodes: {n_non_leaf}')
print(f'Number of leaf nodes: {n_leaf}')
n = 128
dict_128_2 = {'Less than 20%': 0, '20~80%': 0, 'More than 80%': 0}
for count in leaf_points_lst:
    util = count / n
    if util < 0.2:
        dict_128_2['Less than 20%'] += 1
    elif 0.2 <= util < 0.8:
        dict_128_2['20~80%'] += 1
    else:
        dict_128_2['More than 80%'] += 1
print(f'Space utilization: {dict_128_2}')
print(f'Number of sub-tree overlapping among the non-leaf nodes: {n_overlap}')

Height of R-Tree index: 32
Number of non-leaf nodes: 5181
Number of leaf nodes: 2119
Space utilization: {'Less than 20%': 0, '20~80%': 1620, 'More than 80%': 499}
Number of sub-tree overlapping among the non-leaf nodes: 885


In [28]:
# Search the nearest neighbor of the 10 random query points
df1 = pd.DataFrame(columns = ['Random point', 'Nearest Neighbor', 'L2 distance', 'N nodes visited', \
                             'N points calculated', 'Rule 1 (MBR)', 'Rule 2 (Object)', 'Rule 3 (MBR)'])
for i in rand_pt_lst:
    ABL = []
    nodes_visited = 0
    points_calculated = 0
    min_dist = float('inf')
    rule1_pruned = 0
    rule2_pruned = 0
    rule3_pruned = 0
    NN = None
    x = i[0]
    y = i[1]
    search_pt = Point(x, y)
    rtree.search_NN(search_pt)
    # print(f'Number of MBRs pruned by rule 1: {rule1_pruned}')
    # print(f'Number of objects pruned by rule 2: {rule2_pruned}')
    # print(f'Number of MBRs pruned by rule 3: {rule3_pruned}')
    # print(f'Nearest Neighbor: ({NN.x}, {NN.y})')
    tmp_dict = {'Random point': '({:.4f}, {:.4f})'.format(i[0], i[1]), \
                'Nearest Neighbor': '({:.4f}, {:.4f})'.format(NN.x, NN.y), \
                'L2 distance': min_dist, \
                'N nodes visited': nodes_visited, \
                'N points calculated': points_calculated, \
                'Rule 1 (MBR)': rule1_pruned, \
                'Rule 2 (Object)': rule2_pruned, \
                'Rule 3 (MBR)': rule3_pruned}
    df1 = df1.append(tmp_dict, ignore_index = True)
df1

,Random point,Nearest Neighbor,L2 distance,N nodes visited,N points calculated,Rule 1 (MBR),Rule 2 (Object),Rule 3 (MBR)
0,"(116.5730, 40.8370)","(116.6103, 40.7559)",0.089251,33,196,23,19600,0
1,"(115.5199, 40.1895)","(115.4941, 40.0399)",0.151807,32,99,7,9116,0
2,"(115.9087, 39.8949)","(115.8787, 39.8713)",0.038087,36,126,14,15876,0
3,"(117.1614, 39.6779)","(116.9139, 39.7938)",0.273320,42,104,29,10816,0
4,"(116.1181, 40.3739)","(116.1103, 40.0677)",0.306275,32,108,14,11627,0
5,"(115.9277, 40.3912)","(115.8585, 40.0477)",0.350314,39,125,14,15625,0
6,"(117.0852, 39.8717)","(116.9172, 39.8187)",0.176124,55,104,36,10816,0
7,"(116.6688, 40.0864)","(116.1238, 40.0540)",0.545942,32,108,14,11664,0
8,"(117.0414, 39.4457)","(116.8190, 39.6238)",0.284955,32,76,21,5776,0
9,"(117.3865, 40.5965)","(117.3769, 40.6310)",0.035874,32,100,26,5252,1


### R-tree (n = 256, d = 2)

In [17]:
# Insert the points in dataset and build the tree
start_time = time.time()
rtree = RTree(n = 256, d = 2)
for i in data:
    x = i[0]
    y = i[1]
    pt = Point(x, y)
    rtree.insert_point(pt)
time_used = time.time() - start_time
print(time_used)

130.99633288383484


In [18]:
# Initialize global parameters
n_non_leaf = 0
n_leaf = 0
leaf_points_lst = []
n_overlap = 0
max_height = 1

# Update the info of R-tree
rtree.update_info()
rtree.get_height(rtree.root, 1)

# Print result
print(f'Height of R-Tree index: {max_height}')
print(f'Number of non-leaf nodes: {n_non_leaf}')
print(f'Number of leaf nodes: {n_leaf}')
n = 256
dict_256_2 = {'Less than 20%': 0, '20~80%': 0, 'More than 80%': 0}
for count in leaf_points_lst:
    util = count / n
    if util < 0.2:
        dict_256_2['Less than 20%'] += 1
    elif 0.2 <= util < 0.8:
        dict_256_2['20~80%'] += 1
    else:
        dict_256_2['More than 80%'] += 1
print(f'Space utilization: {dict_256_2}')
print(f'Number of sub-tree overlapping among the non-leaf nodes: {n_overlap}')

Height of R-Tree index: 24
Number of non-leaf nodes: 2480
Number of leaf nodes: 1047
Space utilization: {'Less than 20%': 0, '20~80%': 782, 'More than 80%': 265}
Number of sub-tree overlapping among the non-leaf nodes: 403


In [19]:
# Search the nearest neighbor of the 10 random query points
df2 = pd.DataFrame(columns = ['Random point', 'Nearest Neighbor', 'L2 distance', 'N nodes visited', \
                             'N points calculated', 'Rule 1 (MBR)', 'Rule 2 (Object)', 'Rule 3 (MBR)'])
for i in rand_pt_lst:
    ABL = []
    nodes_visited = 0
    points_calculated = 0
    min_dist = float('inf')
    rule1_pruned = 0
    rule2_pruned = 0
    rule3_pruned = 0
    NN = None
    x = i[0]
    y = i[1]
    search_pt = Point(x, y)
    rtree.search_NN(search_pt)
    tmp_dict = {'Random point': '({:.4f}, {:.4f})'.format(i[0], i[1]), \
                'Nearest Neighbor': '({:.4f}, {:.4f})'.format(NN.x, NN.y), \
                'L2 distance': min_dist, \
                'N nodes visited': nodes_visited, \
                'N points calculated': points_calculated, \
                'Rule 1 (MBR)': rule1_pruned, \
                'Rule 2 (Object)': rule2_pruned, \
                'Rule 3 (MBR)': rule3_pruned}
    df2 = df2.append(tmp_dict, ignore_index = True)
df2

,Random point,Nearest Neighbor,L2 distance,N nodes visited,N points calculated,Rule 1 (MBR),Rule 2 (Object),Rule 3 (MBR)
0,"(116.5730, 40.8370)","(116.6103, 40.7559)",0.089251,24,208,16,42904,1
1,"(115.5199, 40.1895)","(115.4941, 40.0399)",0.151807,24,160,7,24394,1
2,"(115.9087, 39.8949)","(116.1965, 39.8448)",0.292145,24,179,7,32041,0
3,"(117.1614, 39.6779)","(116.9139, 39.7938)",0.273320,24,227,22,51529,0
4,"(116.1181, 40.3739)","(116.2078, 39.8943)",0.487963,24,176,6,30976,0
5,"(115.9277, 40.3912)","(116.2050, 39.8934)",0.569810,24,176,6,30976,0
6,"(117.0852, 39.8717)","(117.0206, 40.0376)",0.178078,24,228,10,51984,0
7,"(116.6688, 40.0864)","(116.4325, 39.8944)",0.304496,24,174,12,30276,0
8,"(117.0414, 39.4457)","(116.8190, 39.6238)",0.284955,24,189,21,35721,0
9,"(117.3865, 40.5965)","(117.1003, 40.3134)",0.402529,24,154,15,23716,0


### R-tree (n = 128, d = 5)

In [20]:
# Insert the points in dataset and build the tree
start_time = time.time()
rtree = RTree(n = 128, d = 5)
for i in data:
    x = i[0]
    y = i[1]
    pt = Point(x, y)
    rtree.insert_point(pt)
time_used = time.time() - start_time
print(time_used)

64.57505178451538


In [21]:
# Initialize global parameters
n_non_leaf = 0
n_leaf = 0
leaf_points_lst = []
n_overlap = 0
max_height = 1

# Update the info of R-tree
rtree.update_info()
rtree.get_height(rtree.root, 1)

# Print result
print(f'Height of R-Tree index: {max_height}')
print(f'Number of non-leaf nodes: {n_non_leaf}')
print(f'Number of leaf nodes: {n_leaf}')
# print(leaf_points_lst)
n = 128
dict_128_5 = {'Less than 20%': 0, '20~80%': 0, 'More than 80%': 0}
for count in leaf_points_lst:
    util = count / n
    if util < 0.2:
        dict_128_5['Less than 20%'] += 1
    elif 0.2 <= util < 0.8:
        dict_128_5['20~80%'] += 1
    else:
        dict_128_5['More than 80%'] += 1
print(f'Space utilization: {dict_128_5}')
print(f'Number of sub-tree overlapping among the non-leaf nodes: {n_overlap}')

Height of R-Tree index: 7
Number of non-leaf nodes: 897
Number of leaf nodes: 2162
Space utilization: {'Less than 20%': 0, '20~80%': 1666, 'More than 80%': 496}
Number of sub-tree overlapping among the non-leaf nodes: 2617


In [22]:
# Search the nearest neighbor of the 10 random query points
df3 = pd.DataFrame(columns = ['Random point', 'Nearest Neighbor', 'L2 distance', 'N nodes visited', \
                             'N points calculated', 'Rule 1 (MBR)', 'Rule 2 (Object)', 'Rule 3 (MBR)'])
for i in rand_pt_lst:
    ABL = []
    nodes_visited = 0
    points_calculated = 0
    min_dist = float('inf')
    rule1_pruned = 0
    rule2_pruned = 0
    rule3_pruned = 0
    NN = None
    x = i[0]
    y = i[1]
    search_pt = Point(x, y)
    rtree.search_NN(search_pt)
    tmp_dict = {'Random point': '({:.4f}, {:.4f})'.format(i[0], i[1]), \
                'Nearest Neighbor': '({:.4f}, {:.4f})'.format(NN.x, NN.y), \
                'L2 distance': min_dist, \
                'N nodes visited': nodes_visited, \
                'N points calculated': points_calculated, \
                'Rule 1 (MBR)': rule1_pruned, \
                'Rule 2 (Object)': rule2_pruned, \
                'Rule 3 (MBR)': rule3_pruned}
    df3 = df3.append(tmp_dict, ignore_index = True)
df3

,Random point,Nearest Neighbor,L2 distance,N nodes visited,N points calculated,Rule 1 (MBR),Rule 2 (Object),Rule 3 (MBR)
0,"(116.5730, 40.8370)","(116.4364, 40.6976)",0.195142,7,115,21,13225,0
1,"(115.5199, 40.1895)","(115.6400, 39.9941)",0.229399,7,106,15,11236,0
2,"(115.9087, 39.8949)","(115.9969, 39.9716)",0.116876,11,198,25,20765,0
3,"(117.1614, 39.6779)","(116.9012, 39.6844)",0.260241,8,109,22,11881,0
4,"(116.1181, 40.3739)","(116.1103, 40.0677)",0.306275,12,190,27,16581,0
5,"(115.9277, 40.3912)","(116.0358, 40.0633)",0.345211,11,238,20,28187,2
6,"(117.0852, 39.8717)","(117.0206, 40.0376)",0.178078,7,52,17,2704,0
7,"(116.6688, 40.0864)","(116.6743, 40.0477)",0.039072,7,119,19,13714,0
8,"(117.0414, 39.4457)","(116.8190, 39.6238)",0.284955,7,114,18,12996,0
9,"(117.3865, 40.5965)","(117.3769, 40.6310)",0.035874,7,100,19,5252,1


### R-tree (n = 256, d = 5)

In [23]:
# Insert the points in dataset and build the tree
start_time = time.time()
rtree = RTree(n = 256, d = 5)
for i in data:
    x = i[0]
    y = i[1]
    pt = Point(x, y)
    rtree.insert_point(pt)
time_used = time.time() - start_time
print(time_used)

81.16911149024963


In [24]:
# Initialize global parameters
n_non_leaf = 0
n_leaf = 0
leaf_points_lst = []
n_overlap = 0
max_height = 1

# Update the info of R-tree
rtree.update_info()
rtree.get_height(rtree.root, 1)

# Print result
print(f'Height of R-Tree index: {max_height}')
print(f'Number of non-leaf nodes: {n_non_leaf}')
print(f'Number of leaf nodes: {n_leaf}')
n = 256
dict_256_5 = {'Less than 20%': 0, '20~80%': 0, 'More than 80%': 0}
for count in leaf_points_lst:
    util = count / n
    if util < 0.2:
        dict_256_5['Less than 20%'] += 1
    elif 0.2 <= util < 0.8:
        dict_256_5['20~80%'] += 1
    else:
        dict_256_5['More than 80%'] += 1
print(f'Space utilization: {dict_256_5}')
print(f'Number of sub-tree overlapping among the non-leaf nodes: {n_overlap}')

Height of R-Tree index: 7
Number of non-leaf nodes: 442
Number of leaf nodes: 1081
Space utilization: {'Less than 20%': 0, '20~80%': 870, 'More than 80%': 211}
Number of sub-tree overlapping among the non-leaf nodes: 1342


In [25]:
# Search the nearest neighbor of the 10 random query points
df4 = pd.DataFrame(columns = ['Random point', 'Nearest Neighbor', 'L2 distance', 'N nodes visited', \
                             'N points calculated', 'Rule 1 (MBR)', 'Rule 2 (Object)', 'Rule 3 (MBR)'])
for i in rand_pt_lst:
    ABL = []
    nodes_visited = 0
    points_calculated = 0
    min_dist = float('inf')
    rule1_pruned = 0
    rule2_pruned = 0
    rule3_pruned = 0
    NN = None
    x = i[0]
    y = i[1]
    search_pt = Point(x, y)
    rtree.search_NN(search_pt)
    tmp_dict = {'Random point': '({:.4f}, {:.4f})'.format(i[0], i[1]), \
                'Nearest Neighbor': '({:.4f}, {:.4f})'.format(NN.x, NN.y), \
                'L2 distance': min_dist, \
                'N nodes visited': nodes_visited, \
                'N points calculated': points_calculated, \
                'Rule 1 (MBR)': rule1_pruned, \
                'Rule 2 (Object)': rule2_pruned, \
                'Rule 3 (MBR)': rule3_pruned}
    df4 = df4.append(tmp_dict, ignore_index = True)
df4

,Random point,Nearest Neighbor,L2 distance,N nodes visited,N points calculated,Rule 1 (MBR),Rule 2 (Object),Rule 3 (MBR)
0,"(116.5730, 40.8370)","(116.5577, 40.6952)",0.142626,7,175,10,18634,4
1,"(115.5199, 40.1895)","(115.4941, 40.0399)",0.151807,7,235,15,54250,1
2,"(115.9087, 39.8949)","(116.1011, 39.9022)",0.192567,7,142,14,10011,1
3,"(117.1614, 39.6779)","(116.8985, 39.7384)",0.269742,11,134,28,17956,0
4,"(116.1181, 40.3739)","(116.1335, 40.1325)",0.241895,7,141,11,9870,1
5,"(115.9277, 40.3912)","(115.8474, 40.1299)",0.273351,9,392,15,77890,0
6,"(117.0852, 39.8717)","(116.9172, 39.8185)",0.176248,14,180,35,32400,0
7,"(116.6688, 40.0864)","(116.4471, 40.0662)",0.222559,7,168,17,28224,0
8,"(117.0414, 39.4457)","(116.8190, 39.6238)",0.284955,7,141,12,19881,0
9,"(117.3865, 40.5965)","(117.3769, 40.6310)",0.035874,7,215,14,26591,1
